# Data Collection

In order to begin analyzing stocks we first need to collect data to analyze. We will do this by scraping a specified timeframe of stock information for a specified number of stocks.


## Getting Started
To get started, we must import some libraries:

In [1]:
import sys
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
print("======================")


import yfinance as yf
import pandas as pd

print ("Successfully imported libraries")

Successfully imported libraries


We will begin by specifying a list of tickers we want to scrape:

In [2]:
tickers = ["AMZN", "TSLA", "BABA", "HD", "TM", "NKE", "MCD", "LOW", "SBUX", "JD", "PDD", "BKNG", "GM", "MELI", "TJX", "NIO", "F", "LULU", "HMC", "CMG"]

From here, we have to define some options:

- Interval: This represents the frequency we want to get stock data
- Time Frame (2 options):
    - A)
        - Period: This represents a timeframe in english (1m, 5m, 1d, 1y, ...)
    - B)
        - Start Date: This is the starting date for data scraping
        - End Date: This is the ending date for data scraping

In [3]:
period = '10y'
start_date = None
end_date = None
interval = "1d"

Just a quick check to ensure variables are instantiated correctly:

In [4]:
if tickers == None:
    raise Exception("You must specify a list of tickers to scrape")

if period == None and start_date == None and end_date == None:
            raise Exception("You must specify one timeframe in order to scrape")
        
if period == None and (start_date == None or end_date == None):
    raise Exception("You must specify both ends of the timeframe in order to scrape")

if period not in ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']:
    raise Exception("Please input a valid period")

if interval not in ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']:
    raise Exception("Please input a valid time interval")

if period != None and (start_date != None or end_date != None):
    raise Exception("You can only specify one type of timeframe in order to scrape")

print("Passed!")

Passed!


## Scraping Data
Now that we have defined the scraping parameters, we can actually begin to scrape. We do this by calling the YFinance download method for each stock. However, we have to consider two cases:
- We are using period
- We are using start/end dates


In [5]:
ticker_string = ' '.join(tickers) #convert from list to space-separated string
print(f"Ticker String: \n{ticker_string}")

if period != None: #if using period
    data = yf.download(
        ticker_string,
        period = period,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )
else:
    data = yf.download( #if using start/end dates
        ticker_string,
        start = start_date, 
        end = end_date,
        interval = interval,
        group_by = 'ticker',
        threads = True
    )


data = data.drop([(i, 'Close') for i in tickers], axis=1)
data = data.rename({"Adj Close": "Close"}, axis=1)


data.head()

Ticker String: 
AMZN TSLA BABA HD TM NKE MCD LOW SBUX JD PDD BKNG GM MELI TJX NIO F LULU HMC CMG
[*********************100%***********************]  20 of 20 completed


NIO                        PDD                        ...  \
           Open High Low Close Volume Open High Low Close Volume  ...   
Date                                                              ...   
2011-11-03  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-04  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-07  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-08  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-09  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   

                  HMC                                                 LOW  \
                 Open       High        Low      Close  Volume       Open   
Date                                                                        
2011-11-03  30.200001  30.440001  29.799999  30.265368  820700  21.389999   
2011-11-04  30.340000  30.590000  30.160000  30.414654  358400  21.490000   
2011-11-07  30.200001  30.549999  30.200001  30.374844  266400  21.600000   
2011-11-08  30.150000  30.340000  29.879999  30.126034  532200  22.480000   
2011-11-09  29.670000  29.770000  29.170000  29.100931  836500  22.370001   

                                                       
                 High        Low      Close    Volume  
Date                                                   
2011-11-03  21.650000  21.059999  18.155680  12121000  
2011-11-04  21.660000  21.209999  18.080143  10412100  
2011-11-07  22.320000  21.480000  18.726456  16891300  
2011-11-08  22.799999  22.040001  19.112572  15432200  
2011-11-09  22.830000  22.049999  18.634130  14451500  

[5 rows x 100 columns]

# Adding Technical Indicators

Using the [TA-Lib](https://mrjbq7.github.io/ta-lib/) package, we can broadcast technical indicators across the time series into their own dataframe columns

In [6]:
import talib
!{sys.executable} -m pip install TA-Lib

def computeRSI(df):
    return talib.RSI(df["Close"], timeperiod=14)

def computeUltimateOscillator(df):
    return talib.ULTOSC(df["High"], df["Low"], df["Close"], timeperiod1=7, timeperiod2=14, timeperiod3=28)

def computeBollingerBands(df):
    upperband, middleband, _ = talib.BBANDS(df["Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    return (upperband - middleband) / df["Close"] # need to normalize so value is not dependent on share price

def computeChaikinOscillator(df):
    return talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"], fastperiod=3, slowperiod=10)

def computeNATR(df):
    return talib.NATR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeSMA(df, window):
    return talib.SMA(df["Close"], timeperiod=window) / df["Close"] # need to normalize so value is not dependent on share price

def computeSAR(df):
    return talib.SAR(df["High"], df["Low"], acceleration=0, maximum=0) / df["Close"] # need to normalize so value is not dependent on share price

def computeWilliamsR(df):
    return talib.WILLR(df["High"], df["Low"], df["Close"], timeperiod=14)

def computeAPO(df):
    return talib.APO(df["Close"], fastperiod=12, slowperiod=26, matype=0) / df["Close"]# need to normalize so value is not dependent on share price

def computeROC(df, window):
    return talib.ROCP(df["Close"], timeperiod=window) 


Now, we can apply these functions across the collected timeseries

In [7]:
for ticker in tickers:
    df = data[ticker]
    data[ticker, 'RSI'] = computeRSI(df)
    data[ticker, 'Ultimate'] = computeUltimateOscillator(df)
    data[ticker, 'BandRadius'] = computeBollingerBands(df)
    data[ticker, 'Chaikin'] = computeChaikinOscillator(df)
    data[ticker, 'NATR'] = computeNATR(df)
    data[ticker, 'SMA_5'] = computeSMA(df, 5)
    data[ticker, 'SMA_20'] = computeSMA(df, 20)
    data[ticker, 'SMA_100'] = computeSMA(df, 100)
    data[ticker, 'SAR'] = computeSAR(df)
    data[ticker, 'Williams%R'] = computeWilliamsR(df)
    data[ticker, 'APO'] = computeAPO(df)
    data[ticker, 'ROC_5'] = computeROC(df, 5)
    data[ticker, 'ROC_20'] = computeROC(df, 20)
    data[ticker, 'ROC_100'] = computeROC(df, 100)

data.head(100)

NIO                        PDD                        ...  \
           Open High Low Close Volume Open High Low Close Volume  ...   
Date                                                              ...   
2011-11-03  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-04  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-07  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-08  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2011-11-09  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
...         ...  ...  ..   ...    ...  ...  ...  ..   ...    ...  ...   
2012-03-22  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2012-03-23  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2012-03-26  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2012-03-27  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   
2012-03-28  NaN  NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN  ...   

                 CMG                                                     \
                NATR     SMA_5    SMA_20   SMA_100       SAR Williams%R   
Date                                                                      
2011-11-03       NaN       NaN       NaN       NaN       NaN        NaN   
2011-11-04       NaN       NaN       NaN       NaN  0.970764        NaN   
2011-11-07       NaN       NaN       NaN       NaN  0.970221        NaN   
2011-11-08       NaN       NaN       NaN       NaN  0.968194        NaN   
2011-11-09       NaN  1.014505       NaN       NaN  0.988643        NaN   
...              ...       ...       ...       ...       ...        ...   
2012-03-22  1.336414  0.991051  0.955714       NaN  0.717480  -2.100688   
2012-03-23  1.354675  0.999514  0.965120       NaN  0.722004 -10.194588   
2012-03-26  1.400056  0.982962  0.948452       NaN  0.706381  -2.286242   
2012-03-27  1.404746  0.994765  0.959437       NaN  0.711963 -15.142728   
2012-03-28  1.438965  0.999691  0.965916  0.853884  0.714133 -20.964122   

                                                  
                 APO     ROC_5    ROC_20 ROC_100  
Date                                              
2011-11-03       NaN       NaN       NaN     NaN  
2011-11-04       NaN       NaN       NaN     NaN  
2011-11-07       NaN       NaN       NaN     NaN  
2011-11-08       NaN       NaN       NaN     NaN  
2011-11-09       NaN       NaN       NaN     NaN  
...              ...       ...       ...     ...  
2012-03-22  0.023066  0.027145  0.080998     NaN  
2012-03-23  0.024224  0.011195  0.072505     NaN  
2012-03-26  0.025041  0.026044  0.092083     NaN  
2012-03-27  0.026489  0.020588  0.075033     NaN  
2012-03-28  0.027705  0.009565  0.076547     NaN  

[100 rows x 380 columns]

At this point, we have a 3D nested dataframe, split by tickers and then by columns. To put this data in CSV files, we will create a CSV file for each ticker containing the relevant information.

In [10]:
import os

for ticker in tickers:
    print(f"Saving data for {ticker}")
    
    tickerdata = data[ticker].dropna()
    if not os.path.exists("data"):
        os.mkdir("data")
    tickerdata.to_csv(f'./data/{ticker}.csv') #write to csv file

print("======================")
print("Saved stocks to CSV files. Take a look inside /data for them.")

Saving data for AMZN
Saving data for TSLA
Saving data for BABA
Saving data for HD
Saving data for TM
Saving data for NKE
Saving data for MCD
Saving data for LOW
Saving data for SBUX
Saving data for JD
Saving data for PDD
Saving data for BKNG
Saving data for GM
Saving data for MELI
Saving data for TJX
Saving data for NIO
Saving data for F
Saving data for LULU
Saving data for HMC
Saving data for CMG
Saved stocks to CSV files. Take a look inside /data for them.
